In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler
print('Libraries imported.')

Libraries imported.


## Get density of Indian-Americans residing in New York boroughwise
#### Source: https://en.wikipedia.org/wiki/Indians_in_the_New_York_City_metropolitan_region

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/Indians_in_the_New_York_City_metropolitan_region").text
soup = BeautifulSoup(source, 'html5lib')
table_IA_demog = soup.find('table',{'class' : 'wikitable sortable'}) # Get only the table
for span in table_IA_demog('span'):  # Getting values besides span<> tag. Decompose destroys span tags leaving the values
    span.decompose()

In [3]:
a,b,c,d,e,f=[],[],[],[],[],[]  # list of our column elements
for rows in table_IA_demog.find_all('tr'):
    cells = rows.find_all('td')
    if(len(cells)==6):
        a.append(cells[0].find(text=True))
        b.append(cells[1].find(text=True))
        c.append(cells[2].find(text=True))
        d.append(cells[3].find(text=True))
        e.append(cells[4].find(text=True))
        f.append(cells[5].find(text=True))

In [4]:
rank_df = pd.DataFrame({'Rank':a,
                        'Borough':b,
                        'City':c,
                        'Indian Americans':d,
                        'Density':e,
                        'Percentage':f}) # Append column elements to our dataframe
rank_df = rank_df.astype(str)
rank_df = rank_df.replace('\n','', regex=True)
rank_df.Density=rank_df.Density.str.replace(",","") # Remove commas from numbers. They prevent conversion from str to int/float
rank_df['Indian Americans'] = rank_df['Indian Americans'].str.replace(",","")
rank_df.Density = rank_df.Density.astype(float)
rank_df['Indian Americans'] = rank_df['Indian Americans'].astype(float)
rank_df.drop(index=(len(rank_df)-1), inplace=True)

In [5]:
rank_df.sort_values('Density', ascending=False)

,Rank,Borough,City,Indian Americans,Density,Percentage
0,1,Queens,New York City,144896.0,1326.5,6.2
2,3,Manhattan,New York City,24359.0,1060.9,1.5
3,4,The Bronx,New York City,16748.0,398.6,1.2
1,2,Brooklyn,New York City,25270.0,357.9,1.0
4,5,Staten Island,New York City,6646.0,113.6,1.4


## Get rental two-bedroom median rental rates around Queens neighborhoods
#### Source:  https://www.renthop.com/study/assets/new-york-city-cost-of-living-2017/nyc-2br-median-rent-and-income-table.html

In [6]:
nyc_rent = pd.read_csv('nyc_rent.csv')
queens_rent = nyc_rent[nyc_rent.Borough=='Queens']
queens_rent = queens_rent[["Borough","Neighborhood","Two-Bedroom Median Rent"]]
queens_rent["Two-Bedroom Median Rent"] = queens_rent["Two-Bedroom Median Rent"].str.replace("$","")
queens_rent["Two-Bedroom Median Rent"] = queens_rent["Two-Bedroom Median Rent"].str.replace(",", "")
queens_rent["Two-Bedroom Median Rent"] = queens_rent["Two-Bedroom Median Rent"].astype(float)
queens_rent = queens_rent.sort_values(["Two-Bedroom Median Rent"]).reset_index(drop=True)
queens_rent.head()

,Borough,Neighborhood,Two-Bedroom Median Rent
0,Queens,Whitestone,1950.0
1,Queens,Fresh Meadows-Utopia,1950.0
2,Queens,Pomonok-Flushing Heights-Hillcrest,1975.0
3,Queens,Middle Village,2000.0
4,Queens,Ozone Park,2000.0


#### Exploding the above neighborhood list

In [7]:
def splitDataFrameList(df,target_column,separator):
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [8]:
queens_rent = splitDataFrameList(queens_rent,'Neighborhood',"-")
queens_rent_top20 = queens_rent.head(20)

In [335]:
queens_rent_top20

,Borough,Neighborhood,Two-Bedroom Median Rent
0,Queens,Whitestone,1950.0
1,Queens,Fresh Meadows,1950.0
2,Queens,Utopia,1950.0
3,Queens,Pomonok,1975.0
4,Queens,Flushing Heights,1975.0
5,Queens,Hillcrest,1975.0
6,Queens,Middle Village,2000.0
7,Queens,Ozone Park,2000.0
8,Queens,Glendale,2000.0
9,Queens,South Ozone Park,2000.0


## Get data of Queens neighborhoods
#### Source: newyork_data.json

In [11]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)

In [12]:
print('Shape of queens_data: ', queens_data.shape)
queens_data.head()

Shape of queens_data:  (81, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


#### Merging Queens data from JSON file with top 10 cheapest neighborhoods with regard to rent

In [16]:
queens_rent_location = pd.merge(queens_rent_top20[['Borough','Neighborhood','Two-Bedroom Median Rent']],
                  queens_data[['Neighborhood','Latitude','Longitude']], on='Neighborhood')
queens_rent_location_top10 = queens_rent_location.head(10)
queens_rent_location_top10

,Borough,Neighborhood,Two-Bedroom Median Rent,Latitude,Longitude
0,Queens,Whitestone,1950.0,40.781291,-73.814202
1,Queens,Fresh Meadows,1950.0,40.734394,-73.782713
2,Queens,Utopia,1950.0,40.733500,-73.796717
3,Queens,Pomonok,1975.0,40.734936,-73.804861
4,Queens,Hillcrest,1975.0,40.723825,-73.797603
5,Queens,Middle Village,2000.0,40.716415,-73.881143
6,Queens,Ozone Park,2000.0,40.680708,-73.843203
7,Queens,Glendale,2000.0,40.702762,-73.870742
8,Queens,South Ozone Park,2000.0,40.668550,-73.809865
9,Queens,Kew Gardens Hills,2050.0,40.722578,-73.820878


In [17]:
CLIENT_ID = '35ZAIRXCXM2N0JJWMERNYKEGI2ABUL2GHMLK442FSGHDT0KH' # your Foursquare ID
CLIENT_SECRET = '5NHK2O2WWSKRKFVLMSGGF3MFQLERRFKHYA2DUV01DDSA1A14' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],
                             v['venue']['location']['lng'],v['venue']['categories'][0]['name']
                             ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood_Latitude', 
                  'Neighborhood_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

In [19]:
queens_top10_venues = getNearbyVenues(names=queens_rent_location_top10['Neighborhood'],
                                   latitudes=queens_rent_location_top10['Latitude'],
                                   longitudes=queens_rent_location_top10['Longitude']
                                  )

Whitestone
Fresh Meadows
Utopia
Pomonok
Hillcrest
Middle Village
Ozone Park
Glendale
South Ozone Park
Kew Gardens Hills


#### Get venue category count for the top 10 neighborhoods

In [24]:
venue_counts = queens_top10_venues.Neighborhood.value_counts()
venue_counts_df = pd.DataFrame(venue_counts).reset_index(drop=False)
venue_counts_df.rename(columns={'index':'Neighborhood','Neighborhood':'Venue_Category_Count'}, inplace=True)
venue_counts_df

,Neighborhood,Venue_Category_Count
0,Ozone Park,35
1,Kew Gardens Hills,21
2,Hillcrest,18
3,Utopia,18
4,Fresh Meadows,16
5,Pomonok,15
6,Middle Village,15
7,South Ozone Park,12
8,Whitestone,4
9,Glendale,4


## Extracting crime data from www.brickunderground.com in New York neighborhoods

In [20]:
source2 = requests.get("https://www.brickunderground.com/blog/2014/07/address_report_safest_and_least_safe_neighborhoods").text
soup2 = BeautifulSoup(source2, 'html5lib')
queens_table2 = soup2.find_all('table',{'class':'data-table'})
a,b=[],[]
for item in queens_table2:
    for rows in item.find_all('tr'):
        cells = rows.find_all('td')
        if(len(cells)==2):
            a.append(cells[0].find(text=True))
            b.append(cells[1].find(text=True))

In [25]:
df = pd.DataFrame({'Neighborhood':a, 'Crime_Per_1000':b})
df.Neighborhood[58] = 'Kew Gardens Hills'
df.Neighborhood[43] = 'Whitestone'

### Merging crime data

In [26]:
rent_crime_df = pd.merge(queens_rent_location_top10, df, on='Neighborhood')
rent_crime_df

,Borough,Neighborhood,Two-Bedroom Median Rent,Latitude,Longitude,Crime_Per_1000
0,Queens,Whitestone,1950.0,40.781291,-73.814202,4.83
1,Queens,Pomonok,1975.0,40.734936,-73.804861,3.98
2,Queens,Ozone Park,2000.0,40.680708,-73.843203,14.4
3,Queens,South Ozone Park,2000.0,40.668550,-73.809865,10.7
4,Queens,Kew Gardens Hills,2050.0,40.722578,-73.820878,10.9


### Merging rent, crime, and venue categories count

In [27]:
Final_DF = pd.merge(rent_crime_df, venue_counts_df, on = 'Neighborhood')
Final_DF = Final_DF[['Borough','Neighborhood','Latitude','Longitude',
                     'Two-Bedroom Median Rent','Crime_Per_1000', 'Venue_Category_Count']]
Final_DF

,Borough,Neighborhood,Latitude,Longitude,Two-Bedroom Median Rent,Crime_Per_1000,Venue_Category_Count
0,Queens,Whitestone,40.781291,-73.814202,1950.0,4.83,4
1,Queens,Pomonok,40.734936,-73.804861,1975.0,3.98,15
2,Queens,Ozone Park,40.680708,-73.843203,2000.0,14.4,35
3,Queens,South Ozone Park,40.668550,-73.809865,2000.0,10.7,12
4,Queens,Kew Gardens Hills,40.722578,-73.820878,2050.0,10.9,21


# Scoring and Ranking

#### First we shall normalize the values of Median Rent, Crime Rate and Venue Count. Then, deduct 1 from Median Rent and Crime Rate normalized values since lower the value of these parameters, better its suited.

In [28]:
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(Final_DF[['Two-Bedroom Median Rent','Crime_Per_1000','Venue_Category_Count']])
Final_DF.loc[:,'Two-Bedroom Median Rent':] = scaled_values
Final_DF[['Two-Bedroom Median Rent','Crime_Per_1000']] = 1-Final_DF[['Two-Bedroom Median Rent','Crime_Per_1000']]
Final_DF['Total_Score'] = Final_DF[['Two-Bedroom Median Rent','Crime_Per_1000','Venue_Category_Count']].sum(axis=1)
Final_DF.sort_values('Total_Score', ascending=False)

,Borough,Neighborhood,Latitude,Longitude,Two-Bedroom Median Rent,Crime_Per_1000,Venue_Category_Count,Total_Score
1,Queens,Pomonok,40.734936,-73.804861,0.75,1.000000,0.354839,2.104839
0,Queens,Whitestone,40.781291,-73.814202,1.00,0.918426,0.000000,1.918426
2,Queens,Ozone Park,40.680708,-73.843203,0.50,0.000000,1.000000,1.500000
3,Queens,South Ozone Park,40.668550,-73.809865,0.50,0.355086,0.258065,1.113151
4,Queens,Kew Gardens Hills,40.722578,-73.820878,0.00,0.335893,0.548387,0.884280


## Result

### Neighbourhood Pomonok score the highest with the right blend of low crime rate, low rental rate and various categories of venues, followed closely by Whitestone

In [39]:
address = 'Queens, NY'
geolocator = Nominatim()
location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
latitude = 40.6524927 
longitude = -73.7914214158161
print(latitude, longitude)
#40.6524927 -73.7914214158161
map_Queens = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(Final_DF['Latitude'], Final_DF['Longitude'],
                                           Final_DF['Borough'], Final_DF['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label)
    folium.Circle([lat, lng],radius=1000,popup=label,color='blue',fill=True,fill_color='#3186cc',
                   fill_opacity=0.5).add_to(map_Queens)
map_Queens

C:\ProgramData\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


40.6524927 -73.7914214158161
